In [1]:
import os
import sys
sys.path.append('../')

from settings import config as cfg
from utils import io_utils as iou
from utils import waveform_plot_utils as wpu
from utils import fourier_math_utils as fmu
from settings import period_bounds as pb
from utils import general_display_utils as gdu

import pyaudio
import numpy as np
#import scipio
from scipy.io import wavfile
from scipy.io.wavfile import write
from io import BytesIO
from IPython.display import Audio, display, HTML
import matplotlib.pyplot as plt
from weasyprint import HTML
import sympy
import librosa
import pandas as pd
import re

from IPython.display import Audio
import ipywidgets as widgets

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
import math
import plotly.io as pio

Function that draws each harmonic

In [ ]:
def draw_harmonics(functions, x_min, x_max, y_min, y_max, name, limit_N):
    
    for N, i in enumerate(functions):
    
        # parse the function string using sympy
        t = sympy.symbols('t')
        f = sympy.sympify(i)

        # create a list of t values within the specified range
        t_values = np.linspace(x_min, x_max, 1000)

        # evaluate the function for each value of t
        y_values = [f.evalf(subs={t: value}) for value in t_values]

        # plot the function using matplotlib
        plt.plot(t_values, y_values)

        # set the x and y axis limits
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)

        # add labels and a title to the plot
        plt.xlabel('t [ms]', fontsize=16)
        plt.ylabel('y(t)', fontsize=16)
        plt.title('Plot of the function y(t)', fontsize=16)
        
        if N < limit_N:
            print("y(t) = " + str(f))
            print("y(t) = " + sympy.latex(f))
            plt.savefig("../results/analysed/f_harmonic_" + str(N+1) + "_" + name + ".pdf")
            plt.show()

Function that takes as a string mathematical function and extracts fourier coefficients and frequency

In [ ]:
def extract_a_b_f(function):
    a = b = frequency = 0
    try:
        # Extract the coefficient of the sine term
        match = re.search("(-?\d+\.\d+)[*]sin\((\d+\.\d+)[*]pi[*]t\)", function)
        if match:
            a = float(match.group(1))
            frequency = float(match.group(2))/2
        
        # Extract the coefficient of the cosine term
        match = re.search("(-?\d+\.\d+)[*]cos\((\d+\.\d+)[*]pi[*]t\)", function)
        if match:
            b = float(match.group(1))
            frequency = float(match.group(2))/2*1000
        
    except:
        pass
    return a, b, frequency

Function that produces the sound from the function in string format

In [ ]:
def produce_sound (functions, powers, name, limit_N):
    duration = 1.5
    total_power = np.sum(powers)
    samples_together = 0
    rate = 44100
    for i, func in enumerate(functions):
        t = sympy.symbols('t')
        function = sympy.sympify(func)
        a, b, f = extract_a_b_f(str(function))
        frequency = f
        amplitude = np.sqrt(a*a + b*b) * np.sqrt(powers[i+1] / total_power)
        t = np.linspace(0, duration, int(duration * rate))
        samples = amplitude * np.sin(2 * np.pi * frequency * t)
        if i >= 0:
            samples_together += samples
        if i < limit_N:
            audio = Audio(samples, rate=rate)
            display(audio)
            write(f"../results/analysed/f_harmonic_{i+1}_{name}.wav", rate, samples)

# Loading sounds

In [2]:
files = [os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, name) for name in os.listdir(cfg.PATH_INSTRUMENT_SAMPLES)]
files.sort(key=lambda x: x.lower()) # making sure the order is the same as in period_bounds.py config file
sounds = []

for file in files:
    path = os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, file)
    sound, rate = iou.load_sound(path)
    sounds.append((sound, rate))

/Users/duje/Documents/Lectures/2023_10_15_glazba_kroz_oci_znanstvenika/instruments_fourier_analysis/notebooks/../utils/io_utils.py:46: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, data = wavfile.read(filename)


# Playing sounds

In [3]:
iou.play_audio(files, n_columns=4)

# Plotting waveforms

In [4]:
wpu.plot_waveform(sounds, cfg.WAVEFORM_ZOOM_PERCENTAGES, files)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Extracting period

In [5]:
periods = []
rates = []

one_period_signals, sample_rates = fmu.extract_periods_and_data_rates(sounds)
one_period_audios = iou.export_and_store_one_period_audio(files, one_period_signals, sample_rates)

wpu.plot_waveform(sounds, cfg.WAVEFORM_ZOOM_PERCENTAGES, files, mark_one_period = True)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Calculating Fourier coefficients

In [6]:
fourier_coefficients_per_instrument = []

for one_period_signal, N in zip(one_period_signals, cfg.N_HARMONICS_PER_INSTRUMENT):
    fourier_coefficients_per_instrument.append(fmu.calculate_fourier_coefficients(one_period_signal, N))

# Representing signal as mathematical function

In [7]:
mathematical_representation_of_signal_per_instrument = []

for fourier_coefficients, period_bounds in zip(fourier_coefficients_per_instrument, pb.PERIOD_BOUNDS.values()):
    T = period_bounds[1] - period_bounds[0]
    mathematical_representation_of_signal_per_instrument.append(fmu.get_mathematical_representation_of_signal(fourier_coefficients, T))

gdu.print_mathematical_representation_of_signal(files, mathematical_representation_of_signal_per_instrument)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Reconstruct original signal

In [8]:
reconstructed_signals = []

for one_period_signal, fourier_coefficients in zip(one_period_signals, fourier_coefficients_per_instrument):
    reconstructed_signals.append(fmu.reconstruct_original_signal(one_period_signal, fourier_coefficients))
    
gdu.display_reconstructed_and_original_audio(files, reconstructed_signals, one_period_signals, sample_rates)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Drawing power spectra of harmonics

In [ ]:
relative_harmonic_powers_per_instrument = []

for fourier_coefficients in fourier_coefficients_per_instrument:
    relative_harmonic_powers_per_instrument.append(fmu.calculate_harmonic_power_spectrum(fourier_coefficients))
    
gdu.draw_harmonics_power_spectra(files, relative_harmonic_powers_per_instrument)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

# Plotting individual harmonics

In [ ]:
gdu.plot_individual_harmonics(
    files, mathematical_representation_of_signal_per_instrument
)

GridBox(children=(Checkbox(value=False, description='cello_c3'), Checkbox(value=False, description='clarinet_c…

### Draw individual harmonics and write their function

In [ ]:
#draw_harmonics(functions, 0, 8, -1, 0.5, "cello", limit_N = 30)
#draw_harmonics(functions, 0, 10, -0.8, 0.8, "clarinet", limit_N = 10)
#draw_harmonics(functions, 0, 10, -1, 0.7, "double_bass", limit_N = 30)
#draw_harmonics(functions, 0, 20, -1, 1, "vocal", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "flute", limit_N = 10)
draw_harmonics(functions, 0, 4, -0.07, 0.07, "guitar_nylon", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.6, 0.6, "oboe", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.4, 0.4, "piano", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "piccolo", limit_N = 10)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_alto", limit_N = 10)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_baritone", limit_N = 35)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_soprano", limit_N = 10)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_tenor", limit_N = 18)
#draw_harmonics(functions, 0, 20, -0.1, 0.1, "guitar_metal", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.35, 0.35, "trumpet", limit_N = 15)
#draw_harmonics(functions, 0, 20, -0.9, 0.9, "violin", limit_N = 15)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "trombone", limit_N = 20)

### Find harmonics in sound and their relative power using librosa

In [ ]:
# Load the waveform from a wav file
#waveform, sample_rate = librosa.load('./instrument_samples/cello-c3_16bit.WAV')
waveform, sample_rate = librosa.load('../data/instrument_samples/clarinet-c5_16_bit.WAV')
waveform, sample_rate = librosa.load('../data/instrument_samples/double-bass-c3_16_bit.WAV')

# Compute the power spectrum of the waveform
power_spectrum = np.abs(np.fft.rfft(waveform))**2

# Determine the harmonic frequencies and their relative power
harmonic_frequencies = np.fft.rfftfreq(waveform.size, d=1/sample_rate)
harmonic_powers = power_spectrum / np.sum(power_spectrum)

# Select only frequencies with relative power > 0.001
#harmonics = harmonic_frequencies[harmonic_powers > 0.001]
#powers = harmonic_powers[harmonic_powers > 0.001]
harmonics = harmonic_frequencies[harmonic_frequencies < 1500]
powers_librosa = harmonic_powers[harmonic_frequencies < 1500]

# Plot the power spectrum
plt.plot(harmonics, powers_librosa)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Relative Power')
plt.show()

In [ ]:
df = pd.DataFrame({'frequency': harmonics, 'powers': powers_librosa})
df["harmonic"] = df['frequency']/(1/note_period*1000)
df['harmonic'] = df['harmonic'].astype(int)

In [ ]:
df = df.sort_values('powers', ascending=False)
df.head(20)

# Generate sound of each harmonic

In [ ]:
produce_sound(functions, powers, "sax_baritone", limit_N = 60)